Making an Animation in Python!

Example using AUS2200 radar output. Adapted from Chris Chambers' tutorial. By Kim Reid and ChatGPT

In [ ]:
#SET UP THE NOTEBOOK
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as mcolors
from dask.distributed import Client
from glob import glob

In [ ]:
client=Client()

In [ ]:
client

In [ ]:
#READ IN PRESSURE AND RADAR DATA FROM AUS2200 - This is from the Feb-Mar 2022 flooding analysis
flist_psl=sorted(glob('/g/data/ua8/AUS2200/flood22-coralsea-control/v1-0/10min/psl_*.nc'))
flist_radar=sorted(glob('/g/data/ua8/AUS2200/flood22-coralsea-control/v1-0/10min/reflmax_*.nc'))

In [ ]:
#DEFINE THE REGION AND TIME PERIOD YOU WANT (to save time and memory)
def _preprocess_eaust(ds):
    return ds.sel(lat=slice(-44,-15), lon=slice(143,157), time=slice("2022-02-24","2022-02-28"))
    

In [ ]:
#READ IN VARIABLES
ds_psl = xr.open_mfdataset(flist_psl,parallel=True,combine='nested',concat_dim='time', preprocess=_preprocess_eaust)
ds_radar=xr.open_mfdataset(flist_radar,parallel=True,combine='nested',concat_dim='time', preprocess=_preprocess_eaust)
psl=ds_psl['psl']
radar=ds_radar['reflmax']

In [ ]:
#[OPTIONAL] CONVERT FROM 10 MIN TO 3 HOURLY DATA
hr_psl=psl.resample(time='3h').mean()
hr_radar=radar.resample(time='3h').mean()

In [ ]:
#DEFINE FOR PLOTTING PURPOSES
x=radar.lat
y=radar.lon

In [ ]:
#TEST ON ONE TIMESTEP
fig,ax=plt.subplots(1,1, subplot_kw={'projection': ccrs.PlateCarree()})

#DEFINE RADAR COLORMAP AND COLORBAR
colormap=cm.hsv_r #_r reverses colorbar
vmn=0
vmx=70
interval=5
lvl=np.arange(vmn,vmx+interval,interval) #set your own colormap

cp2=ax.contourf(y,x,hr_radar[10,:,:],cmap=colormap,levels=lvl,vmin=vmn,vmax=vmx,extend='max',transform=ccrs.PlateCarree())
cp1=ax.contour(y,x,hr_psl[10,:,:],colors='gray',linestyles='dashed',linewidths=0.75, transform=ccrs.PlateCarree())

# add colour bar, have to define our tick interval based on same intervals as used in the contour plot.
tcks = np.arange(vmn,vmx+5,5)
cbar = fig.colorbar(cp2, label= 'DBZ', ticks=tcks, shrink=0.8)

ax.coastlines()
plt.show()


In [ ]:
#Make Animation
import matplotlib.animation as animation


# Define the number of time steps
num_hours = hr_radar.shape[0]  # Assuming the data is [time, lat, lon]

# Access the time dimension from the xarray DataArray
times = hr_radar.time  # This assumes 'time' is the dimension name

# Create the figure and map projection 
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.PlateCarree()})

# Define levels and colormap for the contours
vmn = 0
vmx = 70
interval = 5
lvl = np.arange(vmn, vmx + interval, interval)
# Initialize the contour plots as None
cp1, cp2 = None, None

cp1 = ax.contour(y, x, hr_psl[0, :, :], cmap='gray_r', transform=ccrs.PlateCarree())
cp2=ax.contourf(y,x,hr_radar[20,:,:],cmap=colormap,levels=lvl,vmin=vmn,vmax=vmx,extend='max',transform=ccrs.PlateCarree())
tcks = np.arange(vmn,vmx+5,5)
cbar = fig.colorbar(cp2, label= '3hr mean DBZ', ticks=tcks, shrink=0.8, pad=0.05)

ax.coastlines()
# Update function for each frame
def update(frame):
    global cp1, cp2  # Declare the variables as global to update them

    # Remove the previous contour and contourf plots before creating new ones
    if cp1 is not None:
        for c in cp1.collections:
            c.remove()
    if cp2 is not None:
        for c in cp2.collections:
            c.remove()
    
    # Plot data for the current timestep (frame)
    cp1 = ax.contour(y, x, hr_psl[frame, :, :], colors='gray',linestyles='dashed',linewidths=0.75, transform=ccrs.PlateCarree())
    cp2 = ax.contourf(y, x, hr_radar[frame, :, :], cmap=colormap, levels=lvl, vmin=vmn, vmax=vmx, extend='max', transform=ccrs.PlateCarree())

    
  # Add a title using the first 10 characters of the time string (yy-mm-dd)
    time_str = str(times[frame].values)[:10]  # Extract the first 10 characters of the time
    ax.set_title(f"Time: {time_str}")

# Create the animation using the update function
ani = animation.FuncAnimation(fig, update, frames=num_hours, repeat=False)

# Save the animation as an MP4 file (or GIF)
ani.save("<output_directory_and_filename>.mp4", writer='ffmpeg', fps=5)  # Adjust fps (frames per second) as needed
